In [54]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
#from fastdtw import fastdtw
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report




In [ ]:
df = pd.read_csv('signs_all_ts.csv')
df2 = pd.read_csv('signs_flat.csv')

In [38]:
df = df.drop(['Unnamed: 0'], axis=1)

In [39]:
df.ID = df.ID.astype(str)
df['ID'] = df['folder'] + df['label'] + df['ID']

In [40]:
g = df.groupby('ID')
df['ix'] = df.index.values

In [41]:
df['frame'] = g['ix'].rank(method='first')

In [42]:
df['frame'] = df['frame'].astype(int)

In [43]:
y = df[['ID','label']].drop_duplicates()['label']
df.drop(['label', 'folder', 'ix'], axis = 1, inplace=True)

In [10]:
extracted_features = extract_features(df, column_id="ID", column_sort="frame")

Feature Extraction: 100%|███████████████████████████████████████████████████████████████████████| 22/22 [40:13<00:00, 109.69s/it]


In [45]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)

In [47]:
impute(extracted_features)
features_filtered = select_features(extracted_features, np.array(Y))

Feature Selection: 100%|████████████████████████████████████████████████████████████████████| 4884/4884 [00:08<00:00, 603.07it/s]


In [48]:
features_filtered.shape

(2565, 1355)

In [50]:
features_filtered

,roll_R__symmetry_looking__r_0.15000000000000002,roll_R__large_standard_deviation__r_0.25,roll_R__large_standard_deviation__r_0.30000000000000004,x_R__max_langevin_fixed_point__m_3__r_30,roll_R__autocorrelation__lag_4,roll_R__symmetry_looking__r_0.2,roll_R__autocorrelation__lag_3,roll_R__autocorrelation__lag_2,roll_R__autocorrelation__lag_1,roll_R__mean_autocorrelation,...,"fore_R__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20",roll_L__variance,roll_L__standard_deviation,"pitch_L__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5","z_L__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_10","fore_R__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_2","fore_R__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20",y_L__count_above_mean,y_L__count_below_mean,"z_R__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_5"
ID,,,,,,,,,,,,,,,,,,,,,
tctodd1God1.0,1.0,1.0,0.0,0.150841,0.808618,1.0,0.865683,0.925422,0.966553,1.862493,...,0.802581,1.664890e-02,0.129031,0.730884,0.109060,1.725073,0.764654,16.0,41.0,-0.086428
tctodd1God2.0,0.0,1.0,1.0,0.091684,0.819119,0.0,0.874671,0.931802,0.979129,2.231095,...,0.996298,3.905988e-03,0.062498,0.416950,-0.067635,-0.239528,0.968745,21.0,36.0,-0.081553
tctodd1God3.0,0.0,1.0,1.0,0.128342,0.870370,0.0,0.915628,0.956141,0.987376,2.228782,...,0.720843,5.186893e-03,0.072020,0.448191,-0.078581,0.786296,0.689916,21.0,36.0,-0.111739
tctodd1I1.0,0.0,1.0,1.0,0.092799,0.853414,1.0,0.907607,0.959043,0.982189,1.939953,...,0.328722,2.891074e-06,0.001700,0.323721,-0.027660,0.135372,0.318096,32.0,25.0,0.081959
tctodd1I2.0,0.0,1.0,1.0,0.092799,0.783773,0.0,0.840207,0.901912,0.964073,2.113713,...,0.027821,7.718489e-07,0.000879,0.162033,-0.021936,-0.001681,0.028095,35.0,22.0,0.498439
tctodd1I3.0,0.0,1.0,1.0,0.092799,0.875323,0.0,0.922689,0.963856,0.982924,2.169661,...,0.000000,1.139095e-06,0.001067,0.169829,-0.028994,0.000000,0.000000,27.0,30.0,0.266076
tctodd1Norway1.0,1.0,0.0,0.0,0.092799,0.029120,1.0,0.314622,0.590122,0.861242,0.917263,...,0.000000,1.041427e-05,0.003227,0.466678,-0.014140,0.000000,0.000000,20.0,37.0,-0.020089
tctodd1Norway2.0,0.0,1.0,0.0,0.092799,0.690335,1.0,0.768974,0.857448,0.949442,2.075189,...,0.000000,8.663081e-06,0.002943,0.164600,-0.026088,0.000000,0.000000,44.0,13.0,-0.014818
tctodd1Norway3.0,0.0,1.0,1.0,0.092799,0.799939,0.0,0.859437,0.914913,0.968173,2.242658,...,0.000000,4.326559e-06,0.002080,0.183492,-0.050315,0.000000,0.000000,11.0,46.0,-0.060674


In [53]:
X = np.array(features_filtered)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)
clf = svm.SVC(kernel='linear', C=10)
clf.fit(X_train, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_test)
sum(y_pred == y_test)/len(y_test)